In [223]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from bert.tokenization import FullTokenizer
from tensorflow.keras import backend as K
import numpy as np
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity



from sentence_transformers import SentenceTransformer, util

In [153]:
df = pd.read_csv("../data/autovit_data.csv")

df

print(df['transmission'].value_counts())

df

Fata             5929
4x4 (automat)    4415
Spate            1050
4x4 (manual)      770
Name: transmission, dtype: int64


,Unnamed: 0,title,price,make,model,year,mileage,fuel_type,engine_power,engine_capacity,transmission,pollution_standard,gearbox,body_type,country_origin,original_owner,no_accident
0,0,Ford Kuga 2.0 TDCi 4x4 Aut. Individual,10500.0,Ford,Kuga,2015,214 547 km,Diesel,140 CP,1 997 cm3,4x4 (manual),NaN,Automata,SUV,NaN,NaN,NaN
1,1,Audi Q3 2.0 40 TFSI S tronic quattro S Line,28600.0,Audi,Q3,2021,36 000 km,Benzina,190 CP,1 984 cm3,4x4 (automat),Euro 6,Automata,SUV,NaN,NaN,NaN
2,2,Toyota Hilux 4x4 Cabina Dubla Comfort,15900.0,Toyota,Hilux,2014,270 000 km,Diesel,144 CP,2 494 cm3,NaN,Euro 5,Manuala,SUV,NaN,NaN,NaN
3,3,Skoda Octavia Combi Diesel 2.0 TDI DSG Ambition,21652.0,Skoda,Octavia,2020,86 665 km,Diesel,149 CP,1 968 cm3,Fata,NaN,Automata,Combi,NaN,NaN,NaN
4,4,BMW Seria 4 430d AT M Sport,30800.0,BMW,Seria 4,2016,80 000 km,Diesel,258 CP,2 993 cm3,Spate,Euro 6,Automata,Coupe,Germania,Da,Da
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14770,15413,BMW X7 xDrive40d MHEV,133500.0,BMW,X7,2023,10 km,Diesel,340 CP,2 993 cm3,4x4 (automat),NaN,Automata,SUV,NaN,NaN,NaN
14771,15414,Mercedes-Benz Sprinter 516 CDI 906.253,10500.0,Mercedes-Benz,Sprinter,2011,520 000 km,Diesel,163 CP,2 143 cm3,NaN,NaN,Manuala,Monovolum,NaN,NaN,NaN
14772,15415,Volvo V90,64317.0,Volvo,V90,2023,50 km,Hibrid,211 CP,1 969 cm3,4x4 (automat),Euro 6d,Automata,Combi,Suedia,NaN,Da
14773,15416,Fiat 500 1.2 Color Therapy,8000.0,Fiat,500,2013,88 000 km,Benzina,69 CP,1 242 cm3,Fata,Euro 5b,Manuala,Compacta,Belgia,Da,Da


In [154]:
trans_df = df

trans_df #14774

trans_df.dropna(subset=["transmission"],inplace=True)
trans_df #14774

print(trans_df) #12164
df



       Unnamed: 0                                            title     price  \
0               0           Ford Kuga 2.0 TDCi 4x4 Aut. Individual   10500.0   
1               1      Audi Q3 2.0 40 TFSI S tronic quattro S Line   28600.0   
3               3  Skoda Octavia Combi Diesel 2.0 TDI DSG Ambition   21652.0   
4               4                      BMW Seria 4 430d AT M Sport   30800.0   
5               5                    BMW Seria 3 320d Touring Aut.   10300.0   
...           ...                                              ...       ...   
14769       15412                            BMW Seria 3 320d Aut.   15990.0   
14770       15413                            BMW X7 xDrive40d MHEV  133500.0   
14772       15415                                        Volvo V90   64317.0   
14773       15416                       Fiat 500 1.2 Color Therapy    8000.0   
14774       15417                                        Volvo V60   13000.0   

        make    model  year     mileage

,Unnamed: 0,title,price,make,model,year,mileage,fuel_type,engine_power,engine_capacity,transmission,pollution_standard,gearbox,body_type,country_origin,original_owner,no_accident
0,0,Ford Kuga 2.0 TDCi 4x4 Aut. Individual,10500.0,Ford,Kuga,2015,214 547 km,Diesel,140 CP,1 997 cm3,4x4 (manual),NaN,Automata,SUV,NaN,NaN,NaN
1,1,Audi Q3 2.0 40 TFSI S tronic quattro S Line,28600.0,Audi,Q3,2021,36 000 km,Benzina,190 CP,1 984 cm3,4x4 (automat),Euro 6,Automata,SUV,NaN,NaN,NaN
3,3,Skoda Octavia Combi Diesel 2.0 TDI DSG Ambition,21652.0,Skoda,Octavia,2020,86 665 km,Diesel,149 CP,1 968 cm3,Fata,NaN,Automata,Combi,NaN,NaN,NaN
4,4,BMW Seria 4 430d AT M Sport,30800.0,BMW,Seria 4,2016,80 000 km,Diesel,258 CP,2 993 cm3,Spate,Euro 6,Automata,Coupe,Germania,Da,Da
5,5,BMW Seria 3 320d Touring Aut.,10300.0,BMW,Seria 3,2013,284 000 km,Diesel,184 CP,1 995 cm3,Spate,Euro 5,Automata,Combi,Germania,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14769,15412,BMW Seria 3 320d Aut.,15990.0,BMW,Seria 3,2016,190 000 km,Diesel,190 CP,1 995 cm3,Spate,NaN,Automata,Sedan,NaN,NaN,NaN
14770,15413,BMW X7 xDrive40d MHEV,133500.0,BMW,X7,2023,10 km,Diesel,340 CP,2 993 cm3,4x4 (automat),NaN,Automata,SUV,NaN,NaN,NaN
14772,15415,Volvo V90,64317.0,Volvo,V90,2023,50 km,Hibrid,211 CP,1 969 cm3,4x4 (automat),Euro 6d,Automata,Combi,Suedia,NaN,Da
14773,15416,Fiat 500 1.2 Color Therapy,8000.0,Fiat,500,2013,88 000 km,Benzina,69 CP,1 242 cm3,Fata,Euro 5b,Manuala,Compacta,Belgia,Da,Da


In [155]:
has_nan = trans_df['mileage'].isnull().any()

if has_nan:
    print("Column has NaN values")
else:
    print("Column does not have NaN values")

Column does not have NaN values


In [156]:
print(trans_df['mileage'].value_counts())

df['processed_mileage'] = df['mileage'].str[:-3]

df['mileage_INT'] = df['processed_mileage'].str.replace(' ', '')

df['processed_mileage']
df

1 km          215
10 km         119
165 000 km     82
170 000 km     81
188 000 km     80
             ... 
193 035 km      1
52 768 km       1
1 550 km        1
105 623 km      1
17 150 km       1
Name: mileage, Length: 3301, dtype: int64


,Unnamed: 0,title,price,make,model,year,mileage,fuel_type,engine_power,engine_capacity,transmission,pollution_standard,gearbox,body_type,country_origin,original_owner,no_accident,processed_mileage,mileage_INT
0,0,Ford Kuga 2.0 TDCi 4x4 Aut. Individual,10500.0,Ford,Kuga,2015,214 547 km,Diesel,140 CP,1 997 cm3,4x4 (manual),NaN,Automata,SUV,NaN,NaN,NaN,214 547,214547
1,1,Audi Q3 2.0 40 TFSI S tronic quattro S Line,28600.0,Audi,Q3,2021,36 000 km,Benzina,190 CP,1 984 cm3,4x4 (automat),Euro 6,Automata,SUV,NaN,NaN,NaN,36 000,36000
3,3,Skoda Octavia Combi Diesel 2.0 TDI DSG Ambition,21652.0,Skoda,Octavia,2020,86 665 km,Diesel,149 CP,1 968 cm3,Fata,NaN,Automata,Combi,NaN,NaN,NaN,86 665,86665
4,4,BMW Seria 4 430d AT M Sport,30800.0,BMW,Seria 4,2016,80 000 km,Diesel,258 CP,2 993 cm3,Spate,Euro 6,Automata,Coupe,Germania,Da,Da,80 000,80000
5,5,BMW Seria 3 320d Touring Aut.,10300.0,BMW,Seria 3,2013,284 000 km,Diesel,184 CP,1 995 cm3,Spate,Euro 5,Automata,Combi,Germania,NaN,NaN,284 000,284000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14769,15412,BMW Seria 3 320d Aut.,15990.0,BMW,Seria 3,2016,190 000 km,Diesel,190 CP,1 995 cm3,Spate,NaN,Automata,Sedan,NaN,NaN,NaN,190 000,190000
14770,15413,BMW X7 xDrive40d MHEV,133500.0,BMW,X7,2023,10 km,Diesel,340 CP,2 993 cm3,4x4 (automat),NaN,Automata,SUV,NaN,NaN,NaN,10,10
14772,15415,Volvo V90,64317.0,Volvo,V90,2023,50 km,Hibrid,211 CP,1 969 cm3,4x4 (automat),Euro 6d,Automata,Combi,Suedia,NaN,Da,50,50
14773,15416,Fiat 500 1.2 Color Therapy,8000.0,Fiat,500,2013,88 000 km,Benzina,69 CP,1 242 cm3,Fata,Euro 5b,Manuala,Compacta,Belgia,Da,Da,88 000,88000


In [157]:
def approximate_value(value):
    if 0 <= value < 5000:
        return 5000
    elif 5000 <= value < 25000:
        return 25000
    elif 25000 <= value < 50000:
        return 50000
    elif 50000 <= value < 75000:
        return 75000
    elif 75000 <= value < 100000:
        return 100000
    elif 100000 <= value < 125000:
        return 125000
    elif 125000 <= value < 150000:
        return 150000
    elif 150000 <= value < 175000:
        return 175000
    elif 175000 <= value < 200000:
        return 200000
    elif 200000 <= value < 225000:
        return 225000
    elif 225000 <= value < 250000:
        return 250000
    elif 250000 <= value < 275000:
        return 275000
    elif 275000 <= value < 300000:
        return 300000
    elif 300000 <= value < 325000:
        return 325000
    elif 325000 <= value < 350000:
        return 350000
    elif 350000 <= value < 375000:
        return 375000
    elif 375000 <= value < 400000:
        return 400000
    elif 400000 <= value < 425000:
        return 425000
    elif 425000 <= value < 450000:
        return 450000
    elif 450000 <= value < 475000:
        return 475000
    elif 475000 <= value < 500000:
        return 500000
    else:
        return value
    
    
    
    
def transform_values(value):
    if pd.isnull(value):
        return value  # Keep NaN values unchanged
    elif value.startswith('4x4'):
        return '4WD'
    elif value in ['Fata', 'Spate']:
        return '2WD'
    else:
        return value  # Keep other values unchanged

In [158]:
df['mileage_INT'] = df['mileage_INT'].astype(int)
df['final_mileage'] = df['mileage_INT'].apply(approximate_value)
df = df.drop('processed_mileage', axis=1)
df = df.drop('mileage_INT', axis=1)
df

,Unnamed: 0,title,price,make,model,year,mileage,fuel_type,engine_power,engine_capacity,transmission,pollution_standard,gearbox,body_type,country_origin,original_owner,no_accident,final_mileage
0,0,Ford Kuga 2.0 TDCi 4x4 Aut. Individual,10500.0,Ford,Kuga,2015,214 547 km,Diesel,140 CP,1 997 cm3,4x4 (manual),NaN,Automata,SUV,NaN,NaN,NaN,225000
1,1,Audi Q3 2.0 40 TFSI S tronic quattro S Line,28600.0,Audi,Q3,2021,36 000 km,Benzina,190 CP,1 984 cm3,4x4 (automat),Euro 6,Automata,SUV,NaN,NaN,NaN,50000
3,3,Skoda Octavia Combi Diesel 2.0 TDI DSG Ambition,21652.0,Skoda,Octavia,2020,86 665 km,Diesel,149 CP,1 968 cm3,Fata,NaN,Automata,Combi,NaN,NaN,NaN,100000
4,4,BMW Seria 4 430d AT M Sport,30800.0,BMW,Seria 4,2016,80 000 km,Diesel,258 CP,2 993 cm3,Spate,Euro 6,Automata,Coupe,Germania,Da,Da,100000
5,5,BMW Seria 3 320d Touring Aut.,10300.0,BMW,Seria 3,2013,284 000 km,Diesel,184 CP,1 995 cm3,Spate,Euro 5,Automata,Combi,Germania,NaN,NaN,300000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14769,15412,BMW Seria 3 320d Aut.,15990.0,BMW,Seria 3,2016,190 000 km,Diesel,190 CP,1 995 cm3,Spate,NaN,Automata,Sedan,NaN,NaN,NaN,200000
14770,15413,BMW X7 xDrive40d MHEV,133500.0,BMW,X7,2023,10 km,Diesel,340 CP,2 993 cm3,4x4 (automat),NaN,Automata,SUV,NaN,NaN,NaN,5000
14772,15415,Volvo V90,64317.0,Volvo,V90,2023,50 km,Hibrid,211 CP,1 969 cm3,4x4 (automat),Euro 6d,Automata,Combi,Suedia,NaN,Da,5000
14773,15416,Fiat 500 1.2 Color Therapy,8000.0,Fiat,500,2013,88 000 km,Benzina,69 CP,1 242 cm3,Fata,Euro 5b,Manuala,Compacta,Belgia,Da,Da,100000


In [159]:
df.dropna(subset=['transmission'], inplace=True)

df

,Unnamed: 0,title,price,make,model,year,mileage,fuel_type,engine_power,engine_capacity,transmission,pollution_standard,gearbox,body_type,country_origin,original_owner,no_accident,final_mileage
0,0,Ford Kuga 2.0 TDCi 4x4 Aut. Individual,10500.0,Ford,Kuga,2015,214 547 km,Diesel,140 CP,1 997 cm3,4x4 (manual),NaN,Automata,SUV,NaN,NaN,NaN,225000
1,1,Audi Q3 2.0 40 TFSI S tronic quattro S Line,28600.0,Audi,Q3,2021,36 000 km,Benzina,190 CP,1 984 cm3,4x4 (automat),Euro 6,Automata,SUV,NaN,NaN,NaN,50000
3,3,Skoda Octavia Combi Diesel 2.0 TDI DSG Ambition,21652.0,Skoda,Octavia,2020,86 665 km,Diesel,149 CP,1 968 cm3,Fata,NaN,Automata,Combi,NaN,NaN,NaN,100000
4,4,BMW Seria 4 430d AT M Sport,30800.0,BMW,Seria 4,2016,80 000 km,Diesel,258 CP,2 993 cm3,Spate,Euro 6,Automata,Coupe,Germania,Da,Da,100000
5,5,BMW Seria 3 320d Touring Aut.,10300.0,BMW,Seria 3,2013,284 000 km,Diesel,184 CP,1 995 cm3,Spate,Euro 5,Automata,Combi,Germania,NaN,NaN,300000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14769,15412,BMW Seria 3 320d Aut.,15990.0,BMW,Seria 3,2016,190 000 km,Diesel,190 CP,1 995 cm3,Spate,NaN,Automata,Sedan,NaN,NaN,NaN,200000
14770,15413,BMW X7 xDrive40d MHEV,133500.0,BMW,X7,2023,10 km,Diesel,340 CP,2 993 cm3,4x4 (automat),NaN,Automata,SUV,NaN,NaN,NaN,5000
14772,15415,Volvo V90,64317.0,Volvo,V90,2023,50 km,Hibrid,211 CP,1 969 cm3,4x4 (automat),Euro 6d,Automata,Combi,Suedia,NaN,Da,5000
14773,15416,Fiat 500 1.2 Color Therapy,8000.0,Fiat,500,2013,88 000 km,Benzina,69 CP,1 242 cm3,Fata,Euro 5b,Manuala,Compacta,Belgia,Da,Da,100000


In [160]:
df['transmission_Updated'] = df['transmission'].apply(transform_values)
df

,Unnamed: 0,title,price,make,model,year,mileage,fuel_type,engine_power,engine_capacity,transmission,pollution_standard,gearbox,body_type,country_origin,original_owner,no_accident,final_mileage,transmission_Updated
0,0,Ford Kuga 2.0 TDCi 4x4 Aut. Individual,10500.0,Ford,Kuga,2015,214 547 km,Diesel,140 CP,1 997 cm3,4x4 (manual),NaN,Automata,SUV,NaN,NaN,NaN,225000,4WD
1,1,Audi Q3 2.0 40 TFSI S tronic quattro S Line,28600.0,Audi,Q3,2021,36 000 km,Benzina,190 CP,1 984 cm3,4x4 (automat),Euro 6,Automata,SUV,NaN,NaN,NaN,50000,4WD
3,3,Skoda Octavia Combi Diesel 2.0 TDI DSG Ambition,21652.0,Skoda,Octavia,2020,86 665 km,Diesel,149 CP,1 968 cm3,Fata,NaN,Automata,Combi,NaN,NaN,NaN,100000,2WD
4,4,BMW Seria 4 430d AT M Sport,30800.0,BMW,Seria 4,2016,80 000 km,Diesel,258 CP,2 993 cm3,Spate,Euro 6,Automata,Coupe,Germania,Da,Da,100000,2WD
5,5,BMW Seria 3 320d Touring Aut.,10300.0,BMW,Seria 3,2013,284 000 km,Diesel,184 CP,1 995 cm3,Spate,Euro 5,Automata,Combi,Germania,NaN,NaN,300000,2WD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14769,15412,BMW Seria 3 320d Aut.,15990.0,BMW,Seria 3,2016,190 000 km,Diesel,190 CP,1 995 cm3,Spate,NaN,Automata,Sedan,NaN,NaN,NaN,200000,2WD
14770,15413,BMW X7 xDrive40d MHEV,133500.0,BMW,X7,2023,10 km,Diesel,340 CP,2 993 cm3,4x4 (automat),NaN,Automata,SUV,NaN,NaN,NaN,5000,4WD
14772,15415,Volvo V90,64317.0,Volvo,V90,2023,50 km,Hibrid,211 CP,1 969 cm3,4x4 (automat),Euro 6d,Automata,Combi,Suedia,NaN,Da,5000,4WD
14773,15416,Fiat 500 1.2 Color Therapy,8000.0,Fiat,500,2013,88 000 km,Benzina,69 CP,1 242 cm3,Fata,Euro 5b,Manuala,Compacta,Belgia,Da,Da,100000,2WD


In [161]:
df_reviews = pd.read_csv("../data/edmunds_processed_data_all_cars_reviews.csv")

df_reviews

print(df['make'].value_counts())

print("\n=======================================\n")
print(df_reviews['Make'].value_counts())

print("\n=======================================\n")

print(df['model'].value_counts())

print("\n=======================================\n")
print(df_reviews['Model'].value_counts())

Mercedes-Benz    1791
BMW              1754
Volkswagen       1362
Audi             1122
Skoda             747
Ford              662
Dacia             597
Renault           573
Opel              391
Volvo             357
Toyota            344
Hyundai           279
Mazda             255
Porsche           238
Nissan            210
Seat              195
Peugeot           192
Land Rover        173
Citroën           145
Mitsubishi        116
Kia               116
Fiat              101
Honda              84
Jaguar             66
Mini               65
Jeep               63
Lamborghini        52
Suzuki             39
Alfa Romeo         31
Lexus              22
Bentley            12
Chevrolet          10
Name: make, dtype: int64


Ford             20717
Chevrolet        19440
Toyota           18747
Honda            12684
Nissan           11843
Volkswagen        9386
Hyundai           8226
BMW               8060
Mazda             7865
Mercedes-Benz     7375
Lexus             6153
Audi            

In [162]:
df['model_updated'] = df['model'].apply(transform_series)
df

,Unnamed: 0,title,price,make,model,year,mileage,fuel_type,engine_power,engine_capacity,transmission,pollution_standard,gearbox,body_type,country_origin,original_owner,no_accident,final_mileage,transmission_Updated,model_updated
0,0,Ford Kuga 2.0 TDCi 4x4 Aut. Individual,10500.0,Ford,Kuga,2015,214 547 km,Diesel,140 CP,1 997 cm3,4x4 (manual),NaN,Automata,SUV,NaN,NaN,NaN,225000,4WD,Kuga
1,1,Audi Q3 2.0 40 TFSI S tronic quattro S Line,28600.0,Audi,Q3,2021,36 000 km,Benzina,190 CP,1 984 cm3,4x4 (automat),Euro 6,Automata,SUV,NaN,NaN,NaN,50000,4WD,Q3
3,3,Skoda Octavia Combi Diesel 2.0 TDI DSG Ambition,21652.0,Skoda,Octavia,2020,86 665 km,Diesel,149 CP,1 968 cm3,Fata,NaN,Automata,Combi,NaN,NaN,NaN,100000,2WD,Octavia
4,4,BMW Seria 4 430d AT M Sport,30800.0,BMW,Seria 4,2016,80 000 km,Diesel,258 CP,2 993 cm3,Spate,Euro 6,Automata,Coupe,Germania,Da,Da,100000,2WD,4 Series
5,5,BMW Seria 3 320d Touring Aut.,10300.0,BMW,Seria 3,2013,284 000 km,Diesel,184 CP,1 995 cm3,Spate,Euro 5,Automata,Combi,Germania,NaN,NaN,300000,2WD,3 Series
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14769,15412,BMW Seria 3 320d Aut.,15990.0,BMW,Seria 3,2016,190 000 km,Diesel,190 CP,1 995 cm3,Spate,NaN,Automata,Sedan,NaN,NaN,NaN,200000,2WD,3 Series
14770,15413,BMW X7 xDrive40d MHEV,133500.0,BMW,X7,2023,10 km,Diesel,340 CP,2 993 cm3,4x4 (automat),NaN,Automata,SUV,NaN,NaN,NaN,5000,4WD,X7
14772,15415,Volvo V90,64317.0,Volvo,V90,2023,50 km,Hibrid,211 CP,1 969 cm3,4x4 (automat),Euro 6d,Automata,Combi,Suedia,NaN,Da,5000,4WD,V90
14773,15416,Fiat 500 1.2 Color Therapy,8000.0,Fiat,500,2013,88 000 km,Benzina,69 CP,1 242 cm3,Fata,Euro 5b,Manuala,Compacta,Belgia,Da,Da,100000,2WD,500


In [163]:
print(df['body_type'].value_counts())

print("\n=======================================\n")

print(df_reviews['Body_Type'].value_counts())

SUV               4664
Sedan             2607
Combi             1610
Compacta          1249
Monovolum          718
Masina de oras     480
Coupe              426
Cabrio             237
Masina mica        173
Name: body_type, dtype: int64


Sedan                                    40042
SUV                                      21234
Hatchback                                12656
SUV AWD                                   9856
Coupe                                     9437
                                         ...  
SuperCab King Ranch Rwd Flareside SB         1
Regular Cab 4WD LB w/2WT                     1
Extended Cab 4WD 6 ft. SB                    1
Crew Cab 4WD 6 ft. SB                        1
Convertible w/3LT                            1
Name: Body_Type, Length: 986, dtype: int64


In [164]:
has_nan = trans_df['body_type'].isnull().any()

if has_nan:
    print("Column has NaN values")
else:
    print("Column does not have NaN values")

Column does not have NaN values


In [165]:
df_final = df[df['body_type'] != 'Masina de oras']

df_final

,Unnamed: 0,title,price,make,model,year,mileage,fuel_type,engine_power,engine_capacity,transmission,pollution_standard,gearbox,body_type,country_origin,original_owner,no_accident,final_mileage,transmission_Updated,model_updated
0,0,Ford Kuga 2.0 TDCi 4x4 Aut. Individual,10500.0,Ford,Kuga,2015,214 547 km,Diesel,140 CP,1 997 cm3,4x4 (manual),NaN,Automata,SUV,NaN,NaN,NaN,225000,4WD,Kuga
1,1,Audi Q3 2.0 40 TFSI S tronic quattro S Line,28600.0,Audi,Q3,2021,36 000 km,Benzina,190 CP,1 984 cm3,4x4 (automat),Euro 6,Automata,SUV,NaN,NaN,NaN,50000,4WD,Q3
3,3,Skoda Octavia Combi Diesel 2.0 TDI DSG Ambition,21652.0,Skoda,Octavia,2020,86 665 km,Diesel,149 CP,1 968 cm3,Fata,NaN,Automata,Combi,NaN,NaN,NaN,100000,2WD,Octavia
4,4,BMW Seria 4 430d AT M Sport,30800.0,BMW,Seria 4,2016,80 000 km,Diesel,258 CP,2 993 cm3,Spate,Euro 6,Automata,Coupe,Germania,Da,Da,100000,2WD,4 Series
5,5,BMW Seria 3 320d Touring Aut.,10300.0,BMW,Seria 3,2013,284 000 km,Diesel,184 CP,1 995 cm3,Spate,Euro 5,Automata,Combi,Germania,NaN,NaN,300000,2WD,3 Series
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14769,15412,BMW Seria 3 320d Aut.,15990.0,BMW,Seria 3,2016,190 000 km,Diesel,190 CP,1 995 cm3,Spate,NaN,Automata,Sedan,NaN,NaN,NaN,200000,2WD,3 Series
14770,15413,BMW X7 xDrive40d MHEV,133500.0,BMW,X7,2023,10 km,Diesel,340 CP,2 993 cm3,4x4 (automat),NaN,Automata,SUV,NaN,NaN,NaN,5000,4WD,X7
14772,15415,Volvo V90,64317.0,Volvo,V90,2023,50 km,Hibrid,211 CP,1 969 cm3,4x4 (automat),Euro 6d,Automata,Combi,Suedia,NaN,Da,5000,4WD,V90
14773,15416,Fiat 500 1.2 Color Therapy,8000.0,Fiat,500,2013,88 000 km,Benzina,69 CP,1 242 cm3,Fata,Euro 5b,Manuala,Compacta,Belgia,Da,Da,100000,2WD,500


In [166]:
def transform_body_types(value):
    if pd.isnull(value):
        return value  # Keep NaN values unchanged
    elif value in ['Masina mica', 'Compacta']:
        return 'Hatchback'
    elif value in ['Combi']:
        return 'Wagon'
    elif value in ['Cabrio']:
        return 'Cabriolet'
    elif value in ['Monovolum']:
        return 'Van'
    else:
        return value  # Keep other values unchanged


In [167]:
print(df_final[(df_final['make'] == 'Ford') & (df_final['model'] == 'Transit') & (df_final['body_type'] == 'Masina mica')])

print(df_final['body_type'].value_counts())

print("\n=======================================\n")

print(df_reviews['Body_Type'].value_counts())

Empty DataFrame
Columns: [Unnamed: 0, title, price, make, model, year, mileage, fuel_type, engine_power, engine_capacity, transmission, pollution_standard, gearbox, body_type, country_origin, original_owner, no_accident, final_mileage, transmission_Updated, model_updated]
Index: []
SUV            4664
Sedan          2607
Combi          1610
Compacta       1249
Monovolum       718
Coupe           426
Cabrio          237
Masina mica     173
Name: body_type, dtype: int64


Sedan                                    40042
SUV                                      21234
Hatchback                                12656
SUV AWD                                   9856
Coupe                                     9437
                                         ...  
SuperCab King Ranch Rwd Flareside SB         1
Regular Cab 4WD LB w/2WT                     1
Extended Cab 4WD 6 ft. SB                    1
Crew Cab 4WD 6 ft. SB                        1
Convertible w/3LT                            1
Name: Bo

In [168]:
df_final['bodyTypeFinal'] = df_final['body_type']
df_final

C:\Users\MVarzaru\AppData\Local\Temp\ipykernel_27664\520390691.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['bodyTypeFinal'] = df_final['body_type']


,Unnamed: 0,title,price,make,model,year,mileage,fuel_type,engine_power,engine_capacity,...,pollution_standard,gearbox,body_type,country_origin,original_owner,no_accident,final_mileage,transmission_Updated,model_updated,bodyTypeFinal
0,0,Ford Kuga 2.0 TDCi 4x4 Aut. Individual,10500.0,Ford,Kuga,2015,214 547 km,Diesel,140 CP,1 997 cm3,...,NaN,Automata,SUV,NaN,NaN,NaN,225000,4WD,Kuga,SUV
1,1,Audi Q3 2.0 40 TFSI S tronic quattro S Line,28600.0,Audi,Q3,2021,36 000 km,Benzina,190 CP,1 984 cm3,...,Euro 6,Automata,SUV,NaN,NaN,NaN,50000,4WD,Q3,SUV
3,3,Skoda Octavia Combi Diesel 2.0 TDI DSG Ambition,21652.0,Skoda,Octavia,2020,86 665 km,Diesel,149 CP,1 968 cm3,...,NaN,Automata,Combi,NaN,NaN,NaN,100000,2WD,Octavia,Combi
4,4,BMW Seria 4 430d AT M Sport,30800.0,BMW,Seria 4,2016,80 000 km,Diesel,258 CP,2 993 cm3,...,Euro 6,Automata,Coupe,Germania,Da,Da,100000,2WD,4 Series,Coupe
5,5,BMW Seria 3 320d Touring Aut.,10300.0,BMW,Seria 3,2013,284 000 km,Diesel,184 CP,1 995 cm3,...,Euro 5,Automata,Combi,Germania,NaN,NaN,300000,2WD,3 Series,Combi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14769,15412,BMW Seria 3 320d Aut.,15990.0,BMW,Seria 3,2016,190 000 km,Diesel,190 CP,1 995 cm3,...,NaN,Automata,Sedan,NaN,NaN,NaN,200000,2WD,3 Series,Sedan
14770,15413,BMW X7 xDrive40d MHEV,133500.0,BMW,X7,2023,10 km,Diesel,340 CP,2 993 cm3,...,NaN,Automata,SUV,NaN,NaN,NaN,5000,4WD,X7,SUV
14772,15415,Volvo V90,64317.0,Volvo,V90,2023,50 km,Hibrid,211 CP,1 969 cm3,...,Euro 6d,Automata,Combi,Suedia,NaN,Da,5000,4WD,V90,Combi
14773,15416,Fiat 500 1.2 Color Therapy,8000.0,Fiat,500,2013,88 000 km,Benzina,69 CP,1 242 cm3,...,Euro 5b,Manuala,Compacta,Belgia,Da,Da,100000,2WD,500,Compacta


In [169]:
df_final['bodyTypeFinal'] = df_final['body_type'].astype(str)
df_final['newbody'] = df_final['bodyTypeFinal'].apply(transform_body_types)
df_final

print(df_final['body_type'].value_counts())

SUV            4664
Sedan          2607
Combi          1610
Compacta       1249
Monovolum       718
Coupe           426
Cabrio          237
Masina mica     173
Name: body_type, dtype: int64


C:\Users\MVarzaru\AppData\Local\Temp\ipykernel_27664\1610098738.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['bodyTypeFinal'] = df_final['body_type'].astype(str)
C:\Users\MVarzaru\AppData\Local\Temp\ipykernel_27664\1610098738.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['newbody'] = df_final['bodyTypeFinal'].apply(transform_body_types)


In [170]:
df_final

print(df_final['model_updated'].value_counts())

Golf          364
Octavia       363
Passat        301
3 Series      250
A4            243
             ... 
Ampera          1
RS Q3           1
307             1
Scenic RX4      1
408             1
Name: model_updated, Length: 406, dtype: int64


In [171]:

df_final = df_final.drop('bodyTypeFinal', axis=1)


In [172]:
df_final

,Unnamed: 0,title,price,make,model,year,mileage,fuel_type,engine_power,engine_capacity,...,pollution_standard,gearbox,body_type,country_origin,original_owner,no_accident,final_mileage,transmission_Updated,model_updated,newbody
0,0,Ford Kuga 2.0 TDCi 4x4 Aut. Individual,10500.0,Ford,Kuga,2015,214 547 km,Diesel,140 CP,1 997 cm3,...,NaN,Automata,SUV,NaN,NaN,NaN,225000,4WD,Kuga,SUV
1,1,Audi Q3 2.0 40 TFSI S tronic quattro S Line,28600.0,Audi,Q3,2021,36 000 km,Benzina,190 CP,1 984 cm3,...,Euro 6,Automata,SUV,NaN,NaN,NaN,50000,4WD,Q3,SUV
3,3,Skoda Octavia Combi Diesel 2.0 TDI DSG Ambition,21652.0,Skoda,Octavia,2020,86 665 km,Diesel,149 CP,1 968 cm3,...,NaN,Automata,Combi,NaN,NaN,NaN,100000,2WD,Octavia,Wagon
4,4,BMW Seria 4 430d AT M Sport,30800.0,BMW,Seria 4,2016,80 000 km,Diesel,258 CP,2 993 cm3,...,Euro 6,Automata,Coupe,Germania,Da,Da,100000,2WD,4 Series,Coupe
5,5,BMW Seria 3 320d Touring Aut.,10300.0,BMW,Seria 3,2013,284 000 km,Diesel,184 CP,1 995 cm3,...,Euro 5,Automata,Combi,Germania,NaN,NaN,300000,2WD,3 Series,Wagon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14769,15412,BMW Seria 3 320d Aut.,15990.0,BMW,Seria 3,2016,190 000 km,Diesel,190 CP,1 995 cm3,...,NaN,Automata,Sedan,NaN,NaN,NaN,200000,2WD,3 Series,Sedan
14770,15413,BMW X7 xDrive40d MHEV,133500.0,BMW,X7,2023,10 km,Diesel,340 CP,2 993 cm3,...,NaN,Automata,SUV,NaN,NaN,NaN,5000,4WD,X7,SUV
14772,15415,Volvo V90,64317.0,Volvo,V90,2023,50 km,Hibrid,211 CP,1 969 cm3,...,Euro 6d,Automata,Combi,Suedia,NaN,Da,5000,4WD,V90,Wagon
14773,15416,Fiat 500 1.2 Color Therapy,8000.0,Fiat,500,2013,88 000 km,Benzina,69 CP,1 242 cm3,...,Euro 5b,Manuala,Compacta,Belgia,Da,Da,100000,2WD,500,Hatchback


In [173]:
df_final['Power_HP'] = df_final['engine_power'].astype(str).str[:-3]
df_final

,Unnamed: 0,title,price,make,model,year,mileage,fuel_type,engine_power,engine_capacity,...,gearbox,body_type,country_origin,original_owner,no_accident,final_mileage,transmission_Updated,model_updated,newbody,Power_HP
0,0,Ford Kuga 2.0 TDCi 4x4 Aut. Individual,10500.0,Ford,Kuga,2015,214 547 km,Diesel,140 CP,1 997 cm3,...,Automata,SUV,NaN,NaN,NaN,225000,4WD,Kuga,SUV,140
1,1,Audi Q3 2.0 40 TFSI S tronic quattro S Line,28600.0,Audi,Q3,2021,36 000 km,Benzina,190 CP,1 984 cm3,...,Automata,SUV,NaN,NaN,NaN,50000,4WD,Q3,SUV,190
3,3,Skoda Octavia Combi Diesel 2.0 TDI DSG Ambition,21652.0,Skoda,Octavia,2020,86 665 km,Diesel,149 CP,1 968 cm3,...,Automata,Combi,NaN,NaN,NaN,100000,2WD,Octavia,Wagon,149
4,4,BMW Seria 4 430d AT M Sport,30800.0,BMW,Seria 4,2016,80 000 km,Diesel,258 CP,2 993 cm3,...,Automata,Coupe,Germania,Da,Da,100000,2WD,4 Series,Coupe,258
5,5,BMW Seria 3 320d Touring Aut.,10300.0,BMW,Seria 3,2013,284 000 km,Diesel,184 CP,1 995 cm3,...,Automata,Combi,Germania,NaN,NaN,300000,2WD,3 Series,Wagon,184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14769,15412,BMW Seria 3 320d Aut.,15990.0,BMW,Seria 3,2016,190 000 km,Diesel,190 CP,1 995 cm3,...,Automata,Sedan,NaN,NaN,NaN,200000,2WD,3 Series,Sedan,190
14770,15413,BMW X7 xDrive40d MHEV,133500.0,BMW,X7,2023,10 km,Diesel,340 CP,2 993 cm3,...,Automata,SUV,NaN,NaN,NaN,5000,4WD,X7,SUV,340
14772,15415,Volvo V90,64317.0,Volvo,V90,2023,50 km,Hibrid,211 CP,1 969 cm3,...,Automata,Combi,Suedia,NaN,Da,5000,4WD,V90,Wagon,211
14773,15416,Fiat 500 1.2 Color Therapy,8000.0,Fiat,500,2013,88 000 km,Benzina,69 CP,1 242 cm3,...,Manuala,Compacta,Belgia,Da,Da,100000,2WD,500,Hatchback,69


In [189]:
print(df_final['price'].value_counts())

#df_final['priceWithoutComma'] = df_final['price'].astype(str).str[:-2]
df_final['priceWithoutComma'] = df_final['price'].astype(str).str.split('.').str[0]

df_final

9990.0      76
10800.0     59
15900.0     59
2750.0      59
8999.0      58
            ..
51158.0      1
52799.0      1
35767.0      1
40573.0      1
133500.0     1
Name: price, Length: 2436, dtype: int64


,Unnamed: 0,title,price,make,model,year,mileage,fuel_type,engine_power,engine_capacity,...,original_owner,no_accident,final_mileage,transmission_Updated,model_updated,newbody,Power_HP,priceWithoutComma,Combined,finalPrice
0,0,Ford Kuga 2.0 TDCi 4x4 Aut. Individual,10500.0,Ford,Kuga,2015,214 547 km,Diesel,140 CP,1 997 cm3,...,NaN,NaN,225000,4WD,Kuga,SUV,140,10500,2015 Ford Kuga SUV 225000 4WD 140 10500,75000
1,1,Audi Q3 2.0 40 TFSI S tronic quattro S Line,28600.0,Audi,Q3,2021,36 000 km,Benzina,190 CP,1 984 cm3,...,NaN,NaN,50000,4WD,Q3,SUV,190,28600,2021 Audi Q3 SUV 50000 4WD 190 28600,100000
3,3,Skoda Octavia Combi Diesel 2.0 TDI DSG Ambition,21652.0,Skoda,Octavia,2020,86 665 km,Diesel,149 CP,1 968 cm3,...,NaN,NaN,100000,2WD,Octavia,Wagon,149,21652,2020 Skoda Octavia Wagon 100000 2WD 149 21652,75000
4,4,BMW Seria 4 430d AT M Sport,30800.0,BMW,Seria 4,2016,80 000 km,Diesel,258 CP,2 993 cm3,...,Da,Da,100000,2WD,4 Series,Coupe,258,30800,2016 BMW 4 Series Coupe 100000 2WD 258 30800,100000
5,5,BMW Seria 3 320d Touring Aut.,10300.0,BMW,Seria 3,2013,284 000 km,Diesel,184 CP,1 995 cm3,...,NaN,NaN,300000,2WD,3 Series,Wagon,184,10300,2013 BMW 3 Series Wagon 300000 2WD 184 10300,75000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14769,15412,BMW Seria 3 320d Aut.,15990.0,BMW,Seria 3,2016,190 000 km,Diesel,190 CP,1 995 cm3,...,NaN,NaN,200000,2WD,3 Series,Sedan,190,15990,2016 BMW 3 Series Sedan 200000 2WD 190 15990,75000
14770,15413,BMW X7 xDrive40d MHEV,133500.0,BMW,X7,2023,10 km,Diesel,340 CP,2 993 cm3,...,NaN,NaN,5000,4WD,X7,SUV,340,133500,2023 BMW X7 SUV 5000 4WD 340 133500,200000
14772,15415,Volvo V90,64317.0,Volvo,V90,2023,50 km,Hibrid,211 CP,1 969 cm3,...,NaN,Da,5000,4WD,V90,Wagon,211,64317,2023 Volvo V90 Wagon 5000 4WD 211 64317,125000
14773,15416,Fiat 500 1.2 Color Therapy,8000.0,Fiat,500,2013,88 000 km,Benzina,69 CP,1 242 cm3,...,Da,Da,100000,2WD,500,Hatchback,69,8000,2013 Fiat 500 Hatchback 100000 2WD 69 8000,50000


In [213]:
df_final['final_mileage'] = df_final['final_mileage'].astype(str)
df_final['year'] = df_final['year'].astype(str)



In [214]:
df_final['Power_HP'] = df_final['Power_HP'].astype(int)
maxClm = df_final['Power_HP'].max()
maxClm

802

In [215]:
def approximate_price(value):
    if 0 <= value < 500:
        return 500
    elif 500 <= value < 1000:
        return 1000
    elif 1000 <= value < 5000:
        return 5000
    elif 5000 <= value < 10000:
        return 10000
    elif 10000 <= value < 25000:
        return 25000
    elif 25000 <= value < 50000:
        return 50000
    elif 50000 <= value < 75000:
        return 75000
    elif 75000 <= value < 100000:
        return 100000
    elif 100000 <= value < 125000:
        return 125000
    elif 125000 <= value < 150000:
        return 150000
    elif 150000 <= value < 175000:
        return 175000
    elif 175000 <= value < 200000:
        return 200000
    elif 200000 <= value < 225000:
        return 225000
    elif 225000 <= value < 250000:
        return 250000
    elif 250000 <= value < 275000:
        return 275000
    elif 275000 <= value < 300000:
        return 300000
    elif 300000 <= value < 325000:
        return 325000
    elif 325000 <= value < 350000:
        return 350000
    elif 350000 <= value < 375000:
        return 375000
    elif 375000 <= value < 400000:
        return 400000
    else:
        return value
    
def approximate_power(value):
    if 0 <= value < 50:
        return 0
    elif 50 <= value < 100:
        return 50
    elif 100 <= value < 150:
        return 100
    elif 150 <= value < 200:
        return 150
    elif 200 <= value < 250:
        return 200
    elif 250 <= value < 300:
        return 250
    elif 300 <= value < 350:
        return 300
    elif 350 <= value < 400:
        return 350
    elif 400 <= value < 450:
        return 400
    elif 450 <= value < 500:
        return 450
    elif 500 <= value < 550:
        return 500
    elif 550 <= value < 600:
        return 550
    elif 600 <= value < 650:
        return 600
    elif 650 <= value < 700:
        return 650
    elif 700 <= value < 750:
        return 700
    elif 750 <= value < 800:
        return 750
    elif 800 <= value < 850:
        return 800
    elif 850 <= value < 900:
        return 850
    else:
        return value

In [216]:
df_final['priceWithoutComma'] = df_final['priceWithoutComma'].astype(int)
df_final['finalPrice'] = df_final['priceWithoutComma'].apply(approximate_price)
df_final

,Unnamed: 0,title,price,make,model,year,mileage,fuel_type,engine_power,engine_capacity,...,no_accident,final_mileage,transmission_Updated,model_updated,newbody,Power_HP,priceWithoutComma,Combined,finalPrice,finalPower
0,0,Ford Kuga 2.0 TDCi 4x4 Aut. Individual,10500.0,Ford,Kuga,2015,214 547 km,Diesel,140 CP,1 997 cm3,...,NaN,225000,4WD,Kuga,SUV,140,10500,2015 Ford Kuga SUV 225000 4WD 140 25000,25000,100
1,1,Audi Q3 2.0 40 TFSI S tronic quattro S Line,28600.0,Audi,Q3,2021,36 000 km,Benzina,190 CP,1 984 cm3,...,NaN,50000,4WD,Q3,SUV,190,28600,2021 Audi Q3 SUV 50000 4WD 190 50000,50000,150
3,3,Skoda Octavia Combi Diesel 2.0 TDI DSG Ambition,21652.0,Skoda,Octavia,2020,86 665 km,Diesel,149 CP,1 968 cm3,...,NaN,100000,2WD,Octavia,Wagon,149,21652,2020 Skoda Octavia Wagon 100000 2WD 149 25000,25000,100
4,4,BMW Seria 4 430d AT M Sport,30800.0,BMW,Seria 4,2016,80 000 km,Diesel,258 CP,2 993 cm3,...,Da,100000,2WD,4 Series,Coupe,258,30800,2016 BMW 4 Series Coupe 100000 2WD 258 50000,50000,250
5,5,BMW Seria 3 320d Touring Aut.,10300.0,BMW,Seria 3,2013,284 000 km,Diesel,184 CP,1 995 cm3,...,NaN,300000,2WD,3 Series,Wagon,184,10300,2013 BMW 3 Series Wagon 300000 2WD 184 25000,25000,150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14769,15412,BMW Seria 3 320d Aut.,15990.0,BMW,Seria 3,2016,190 000 km,Diesel,190 CP,1 995 cm3,...,NaN,200000,2WD,3 Series,Sedan,190,15990,2016 BMW 3 Series Sedan 200000 2WD 190 25000,25000,150
14770,15413,BMW X7 xDrive40d MHEV,133500.0,BMW,X7,2023,10 km,Diesel,340 CP,2 993 cm3,...,NaN,5000,4WD,X7,SUV,340,133500,2023 BMW X7 SUV 5000 4WD 340 150000,150000,300
14772,15415,Volvo V90,64317.0,Volvo,V90,2023,50 km,Hibrid,211 CP,1 969 cm3,...,Da,5000,4WD,V90,Wagon,211,64317,2023 Volvo V90 Wagon 5000 4WD 211 75000,75000,200
14773,15416,Fiat 500 1.2 Color Therapy,8000.0,Fiat,500,2013,88 000 km,Benzina,69 CP,1 242 cm3,...,Da,100000,2WD,500,Hatchback,69,8000,2013 Fiat 500 Hatchback 100000 2WD 69 10000,10000,50


In [217]:
df_final['Power_HP'] = df_final['Power_HP'].astype(int)
df_final['finalPower'] = df_final['Power_HP'].apply(approximate_power).astype(str)
df_final

,Unnamed: 0,title,price,make,model,year,mileage,fuel_type,engine_power,engine_capacity,...,no_accident,final_mileage,transmission_Updated,model_updated,newbody,Power_HP,priceWithoutComma,Combined,finalPrice,finalPower
0,0,Ford Kuga 2.0 TDCi 4x4 Aut. Individual,10500.0,Ford,Kuga,2015,214 547 km,Diesel,140 CP,1 997 cm3,...,NaN,225000,4WD,Kuga,SUV,140,10500,2015 Ford Kuga SUV 225000 4WD 140 25000,25000,100
1,1,Audi Q3 2.0 40 TFSI S tronic quattro S Line,28600.0,Audi,Q3,2021,36 000 km,Benzina,190 CP,1 984 cm3,...,NaN,50000,4WD,Q3,SUV,190,28600,2021 Audi Q3 SUV 50000 4WD 190 50000,50000,150
3,3,Skoda Octavia Combi Diesel 2.0 TDI DSG Ambition,21652.0,Skoda,Octavia,2020,86 665 km,Diesel,149 CP,1 968 cm3,...,NaN,100000,2WD,Octavia,Wagon,149,21652,2020 Skoda Octavia Wagon 100000 2WD 149 25000,25000,100
4,4,BMW Seria 4 430d AT M Sport,30800.0,BMW,Seria 4,2016,80 000 km,Diesel,258 CP,2 993 cm3,...,Da,100000,2WD,4 Series,Coupe,258,30800,2016 BMW 4 Series Coupe 100000 2WD 258 50000,50000,250
5,5,BMW Seria 3 320d Touring Aut.,10300.0,BMW,Seria 3,2013,284 000 km,Diesel,184 CP,1 995 cm3,...,NaN,300000,2WD,3 Series,Wagon,184,10300,2013 BMW 3 Series Wagon 300000 2WD 184 25000,25000,150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14769,15412,BMW Seria 3 320d Aut.,15990.0,BMW,Seria 3,2016,190 000 km,Diesel,190 CP,1 995 cm3,...,NaN,200000,2WD,3 Series,Sedan,190,15990,2016 BMW 3 Series Sedan 200000 2WD 190 25000,25000,150
14770,15413,BMW X7 xDrive40d MHEV,133500.0,BMW,X7,2023,10 km,Diesel,340 CP,2 993 cm3,...,NaN,5000,4WD,X7,SUV,340,133500,2023 BMW X7 SUV 5000 4WD 340 150000,150000,300
14772,15415,Volvo V90,64317.0,Volvo,V90,2023,50 km,Hibrid,211 CP,1 969 cm3,...,Da,5000,4WD,V90,Wagon,211,64317,2023 Volvo V90 Wagon 5000 4WD 211 75000,75000,200
14773,15416,Fiat 500 1.2 Color Therapy,8000.0,Fiat,500,2013,88 000 km,Benzina,69 CP,1 242 cm3,...,Da,100000,2WD,500,Hatchback,69,8000,2013 Fiat 500 Hatchback 100000 2WD 69 10000,10000,50


In [220]:
df_final['finalPrice'] = df_final['finalPrice'].astype(str)

df_final['Combined'] = df_final[['year', 'make', 'model_updated', 'newbody', 'final_mileage', 'transmission_Updated', 'finalPrice', 'finalPower']].agg(' '.join, axis=1)
df_final

,Unnamed: 0,title,price,make,model,year,mileage,fuel_type,engine_power,engine_capacity,...,no_accident,final_mileage,transmission_Updated,model_updated,newbody,Power_HP,priceWithoutComma,Combined,finalPrice,finalPower
0,0,Ford Kuga 2.0 TDCi 4x4 Aut. Individual,10500.0,Ford,Kuga,2015,214 547 km,Diesel,140 CP,1 997 cm3,...,NaN,225000,4WD,Kuga,SUV,140,10500,2015 Ford Kuga SUV 225000 4WD 25000 100,25000,100
1,1,Audi Q3 2.0 40 TFSI S tronic quattro S Line,28600.0,Audi,Q3,2021,36 000 km,Benzina,190 CP,1 984 cm3,...,NaN,50000,4WD,Q3,SUV,190,28600,2021 Audi Q3 SUV 50000 4WD 50000 150,50000,150
3,3,Skoda Octavia Combi Diesel 2.0 TDI DSG Ambition,21652.0,Skoda,Octavia,2020,86 665 km,Diesel,149 CP,1 968 cm3,...,NaN,100000,2WD,Octavia,Wagon,149,21652,2020 Skoda Octavia Wagon 100000 2WD 25000 100,25000,100
4,4,BMW Seria 4 430d AT M Sport,30800.0,BMW,Seria 4,2016,80 000 km,Diesel,258 CP,2 993 cm3,...,Da,100000,2WD,4 Series,Coupe,258,30800,2016 BMW 4 Series Coupe 100000 2WD 50000 250,50000,250
5,5,BMW Seria 3 320d Touring Aut.,10300.0,BMW,Seria 3,2013,284 000 km,Diesel,184 CP,1 995 cm3,...,NaN,300000,2WD,3 Series,Wagon,184,10300,2013 BMW 3 Series Wagon 300000 2WD 25000 150,25000,150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14769,15412,BMW Seria 3 320d Aut.,15990.0,BMW,Seria 3,2016,190 000 km,Diesel,190 CP,1 995 cm3,...,NaN,200000,2WD,3 Series,Sedan,190,15990,2016 BMW 3 Series Sedan 200000 2WD 25000 150,25000,150
14770,15413,BMW X7 xDrive40d MHEV,133500.0,BMW,X7,2023,10 km,Diesel,340 CP,2 993 cm3,...,NaN,5000,4WD,X7,SUV,340,133500,2023 BMW X7 SUV 5000 4WD 150000 300,150000,300
14772,15415,Volvo V90,64317.0,Volvo,V90,2023,50 km,Hibrid,211 CP,1 969 cm3,...,Da,5000,4WD,V90,Wagon,211,64317,2023 Volvo V90 Wagon 5000 4WD 75000 200,75000,200
14773,15416,Fiat 500 1.2 Color Therapy,8000.0,Fiat,500,2013,88 000 km,Benzina,69 CP,1 242 cm3,...,Da,100000,2WD,500,Hatchback,69,8000,2013 Fiat 500 Hatchback 100000 2WD 10000 50,10000,50


In [222]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df_final['Combined'])

tfidf_matrix

<11684x475 sparse matrix of type '<class 'numpy.float64'>'
	with 94934 stored elements in Compressed Sparse Row format>